In [3]:
!pip3 install networkx
!pip3 install -U nltk
!pip3 install rouge


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge import Rouge
import networkx as nx
import sys
import nltk
sys.setrecursionlimit(10**6)

In [ ]:

nltk.download('stopwords')
nltk.download('punkt')

# Read the dataset from CSV
df1 = pd.read_csv('/content/drive/MyDrive/NLP/data/data3.csv')
df2 = pd.read_csv('/content/drive/MyDrive/NLP/data/data7.csv')

# Concatenate the dataframes vertically
data = pd.concat([df1, df2], axis=0)

data.dropna(subset=['title', 'abstract'], inplace=True)
# Reset the index of the combined dataframe
data.reset_index(drop=True, inplace=True)
data = data.head(2000)
# Preprocessing
def preprocess_text(text):
    if isinstance(text, str):  # Check if the text is a valid string
        sentences = sent_tokenize(text)
        return sentences
    return []

data['preprocessed_abstract'] = data['abstract'].apply(preprocess_text)

# Filter out empty or missing abstracts
data = data[data['preprocessed_abstract'].apply(len) > 0]

# Select a subset of data points


# Sentence Similarity
vectorizer = TfidfVectorizer()
sentence_vectors = vectorizer.fit_transform(data['preprocessed_abstract'].apply(' '.join))
similarity_matrix = cosine_similarity(sentence_vectors)

# Graph Construction
graph = nx.from_numpy_array(similarity_matrix)

# Graph Ranking (PageRank)
scores = nx.pagerank(graph)

# Calculate sentence scores
sentence_scores = {i: score for i, score in enumerate(scores)}

# Set the ratio of sentences to include in the summary
summary_ratio = 0.3  # Adjust as needed

# Select Top Sentences based on scores
num_sentences = int(len(data) * summary_ratio)
top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
summary = ' '.join([data['preprocessed_abstract'][i][0] for i in top_sentences])  # Use [0] to get the first sentence

# Extract the corresponding titles for evaluation
titles = [data['title'][i] for i in top_sentences]

# Compute ROUGE score
rouge = Rouge()
rouge_scores = rouge.get_scores(summary, ' '.join(titles))

# Extract the ROUGE-1 F1 score for evaluation
rouge_1_f1_score = rouge_scores[0]['rouge-1']['f']

print("Generated Summary:", summary)
print("ROUGE-1 F1 Score:", rouge_1_f1_score)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
